In [ ]:
import os
from dotenv import load_dotenv
import asyncio

from polymarket.src.stream.polymarket import WebSocketListener
from polymarket.src.stream.binance import binance_price_ws
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from polymarket.src.db.ingest_stream import insert_into_db, parse_message
from polymarket.src.config.logger import logger

In [ ]:
conf = (
    'http::addr=localhost:9000;'
    'username=admin;password=quest;'
    'auto_flush_rows=100;auto_flush_interval=1000;'
)
table_name = "trade_stream"
symbols = ["btcusdt"]
queue = asyncio.Queue()
MARKET_CHANNEL = "market"
USER_CHANNEL = "user"
load_dotenv("../../.env")

url = os.getenv("POLYMARKET_WEBSOCKET_URL")

asset_ids = [
    "115172850081268399385975128465963399335156000901458077700537686348329692537170",
]


async def main():
    loop = asyncio.get_running_loop()
    listener = WebSocketListener(url, asset_ids, queue=queue, loop=loop, market_channel=MARKET_CHANNEL)
    executor = ProcessPoolExecutor(max_workers=10)
    producers = [
        asyncio.create_task(binance_price_ws(queue=queue, symbols=symbols)),
        asyncio.create_task(listener.run()),
    ]
    workers = [asyncio.create_task(insert_into_db(conf, table_name, queue, parse_message, executor, logger))]

    await asyncio.gather(*producers, *workers)

await main()

[{'asks': [{'price': '0.99', 'size': '108765.05'},
           {'price': '0.98', 'size': '6016.58'},
           {'price': '0.97', 'size': '30935'},
           {'price': '0.96', 'size': '259.73'},
           {'price': '0.95', 'size': '2740'},
           {'price': '0.94', 'size': '15900'},
           {'price': '0.93', 'size': '1595.71'},
           {'price': '0.92', 'size': '225'},
           {'price': '0.91', 'size': '7.16'},
           {'price': '0.9', 'size': '13359.27'},
           {'price': '0.89', 'size': '20'},
           {'price': '0.88', 'size': '50'},
           {'price': '0.87', 'size': '100'},
           {'price': '0.85', 'size': '100'},
           {'price': '0.84', 'size': '12.5'},
           {'price': '0.82', 'size': '50'},
           {'price': '0.8', 'size': '6165.2'},
           {'price': '0.79', 'size': '5'},
           {'price': '0.75', 'size': '25.04'},
           {'price': '0.7', 'size': '15'},
           {'price': '0.66', 'size': '12'},
           {'price': '0.65', 's